# Set up

## Import

In [4]:
import pandas as pd
import numpy as np
import os

import warnings
from tqdm import tqdm

# 차로변경차량 정보 추출

## 양양방향

### Directory

In [ ]:
file_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/27_남양주영업소_드론영상분석/양양방향/01_density_by_target_LOS_concated_lanechange'

In [ ]:
save_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/차로변경차량정보'

In [ ]:
file_list = os.listdir(file_dir)
file_list = file_list[3:]
print(file_list)

### Extract
* 차로변경 차량별로 현재차로-목표차로 페어를 입력하기
* 현재차로-목표차로 페어에 따른 목표차로 후행차량(FV) 정보 구하기
* `df.at[idx, 'Lane Identification Past'] = str(Lane_past)`

In [ ]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

for file in file_list:
    
    file_path = os.path.join(file_dir, file)
    df = pd.read_csv(file_path)
    
    df2 = pd.DataFrame()
    
    # 차로변경을 한 차량 리스트만 뽑기
    LC_veh_list = df[df['Lane_change'] == 'Change']['Vehicle ID'].unique()
    
    for veh in LC_veh_list: # 차로변경을 한 각 차량별로
        df_veh = df[df['Vehicle ID'] == veh] ## 각 차량별 데이터프레임
        df_veh['Lane_record_split'] = df_veh['Lane_record_split'].backfill()
        df_veh['Lane_present'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[0]
        df_veh['Lane_target'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[1]
        
        df_veh['Lane_change_intend'] = df_veh['Lane_change_direction'].copy()
        df_veh['Lane_change_intend'] = df_veh['Lane_change_intend'].backfill()
    
        df2 = pd.concat([df2, df_veh])
    
    df2.dropna(subset = ['Lane_record_split'], inplace = True) # Lane_record_split이 nan인 행 드랍
    
    save_file = f'LC_vehicle_Lane_info_{file[:4]}.csv'
    save_path = os.path.join(save_dir, save_file)
    
    df2.to_csv(save_path)

## 서울방향

### Directory

In [ ]:
file_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/27_남양주영업소_드론영상분석/서울방향/01_density_by_target_LOS_concated_lanechange'

In [ ]:
file_list = os.listdir(file_dir)
file_list = file_list[:6]
print(file_list)

### Extract
* 차로변경 차량별로 현재차로-목표차로 페어를 입력하기
* 현재차로-목표차로 페어에 따른 목표차로 후행차량(FV) 정보 구하기
* `df.at[idx, 'Lane Identification Past'] = str(Lane_past)`

In [ ]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

for file in file_list:
    
    file_path = os.path.join(file_dir, file)
    df = pd.read_csv(file_path)
    
    df2 = pd.DataFrame()
    
    # 차로변경을 한 차량 리스트만 뽑기
    LC_veh_list = df[df['Lane_change'] == 'Change']['Vehicle ID'].unique()
    
    for veh in LC_veh_list: # 차로변경을 한 각 차량별로
        df_veh = df[df['Vehicle ID'] == veh] ## 각 차량별 데이터프레임
        df_veh['Lane_record_split'] = df_veh['Lane_record_split'].backfill()
        df_veh['Lane_present'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[0]
        df_veh['Lane_target'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[1]
        
        df_veh['Lane_change_intend'] = df_veh['Lane_change_direction'].copy()
        df_veh['Lane_change_intend'] = df_veh['Lane_change_intend'].backfill()
    
        df2 = pd.concat([df2, df_veh])
    
    df2.dropna(subset = ['Lane_record_split'], inplace = True) # Lane_record_split이 nan인 행 드랍
        
    save_file = f'LC_vehicle_Lane_info_{file[:4]}.csv'
    save_path = os.path.join(save_dir, save_file)
    
    df2.to_csv(save_path)

## 서울TG 부산방향
* 22.10.24 추가
* 김영준으로부터 전달받은 서울TG 부산방향 통합본 자료 가공

### 데이터 로드

In [1]:
SeoulTG = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/raw_Seoul_TG.csv'

In [2]:
changeTG = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/Change_Seoul_TG.csv'

In [5]:
df = pd.read_csv(SeoulTG)

### 전처리 : 차로변경 Change 설정

In [6]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

df['Lane_record'] = None # 차량의 차로변경기록
df['Lane Identification Past'] = None # 차량의 직전 차로 위치. 변경되었을 경우 Lane Identification과 불일치하게 된다. ############## 22.10.22 추가함
df['Lane_record_split'] = None # 그때그때 차로변경 기록. ###################### 22.10.22  추가함
    
df['Lane_00'] = None # 차량의 첫 차로
df['Lane_99'] = None # 차량의 마지막 차로
df['Lane_change'] = None # 차량이 차로변경을 했을 시 'Change' 기록 
df['Lane_change_direction'] = None # 차량의 차로변경 방향. 'Left' or 'Right' 기록 ######################## 22.10.21 추가함
    
if len(df) > 0:

    veh_list = df['Vehicle ID'].unique() # 차량 리스트 뽑기
    
    df = df.sort_values(by = 'Frame ID') # 데이터프레임을 Frame ID에 따라서 오름차순 정렬
    
    for veh in tqdm(veh_list): # 각 차량별로
        Lane_00 = df[df['Vehicle ID'] == veh]['Lane Identification'].iloc[0] # 차량이 처음 진행하던 차로
        Lane_99 = df[df['Vehicle ID'] == veh]['Lane Identification'].iloc[-1] # 차량이 마지막 진행하던 차로
        
        veh_frame_list = df[df['Vehicle ID'] == veh]['Frame ID'].unique() # 해당 차량이 주행하는 동안의 프레임 리스트
        
        Lane_past = Lane_00 # Lane_past(각 차량 현재차로)를 초기화
        Lane_record = Lane_00 # 차로변경기록 초기화
        Lane_present = Lane_00 # 현재차로 초기화
        
        for frm in veh_frame_list : # 각 차량의 각 프레임별로
            
            Lane_past = Lane_present # iteration 돌기 전의 현재차로가 과거차로로 할당
            Lane_present = df[(df['Vehicle ID'] == veh) & (df['Frame ID'] == frm)]['Lane Identification'].iloc[0] # 현재 차량이 속한 차로를 할당            
                
            if Lane_past == Lane_present: # 현재차로와 직전차로가 같으면 아무 문제 없음
                pass
            
            else: # 현재차로와 직전 차로가 다를 시
                idx = df[(df['Vehicle ID'] == veh) & (df['Frame ID'] == frm)]['Lane_change'].index[0] # 차로변경된 행의 인덱스
                    
                df.at[idx, 'Lane Identification Past'] = str(Lane_past) ################ 22.10.22 추가
                df.at[idx, 'Lane_record_split'] = str(Lane_past) + '_' + str(Lane_present) ################ 22.10.22 추가
                    
                df.at[idx, 'Lane_change'] = 'Change' # 차로변경되자마자 셀에 'Change' 표시
                Lane_record = str(Lane_record) + '_' + str(Lane_present)
                    
                ########################################## 22.10.21 추가  : 차로변경 방향 컬럼 추가 ##########################################
                    
                if (Lane_past[-1] != 'N') and (Lane_present[-1] != 'N'): # Lane_past, Lane_present가 Unknown이 아니면 
                    Lane_past_num = int(Lane_past[-1])
                    Lane_present_num = int(Lane_present[-1])
                    
                    if Lane_past_num > Lane_present_num: # 이 경우 왼쪽으로(고속차로로) 이동한 것임
                        df.at[idx, 'Lane_change_direction'] = 'Left'
                    
                    elif Lane_past_num < Lane_present_num: # 이 경우 오른쪽으로(저속차로로) 이동한 것임
                        df.at[idx, 'Lane_change_direction'] = 'Right'
                    
                    else:
                        pass
                    
                else: # Lane_past, Lane_present 중 하나 이상이 Unknown이면 일단 방향 패스. 이건 수동으로 지정해줘야 한다.
                    pass
            ###############################################################################################################################
               
        df.loc[df['Vehicle ID'] == veh, 'Lane_00'] = Lane_00 # 차량별 초기 차로번호
        df.loc[df['Vehicle ID'] == veh, 'Lane_99'] = Lane_99 # 차량별 마지막 차로번호
        df.loc[df['Vehicle ID'] == veh, 'Lane_record'] = Lane_record # 차량별 차로변경 기록 입력

    df.to_csv(changeTG, encoding = 'cp949')
            
else:
    pass

100%|████████████████████████████████████████████████████████████████████████████████| 759/759 [00:48<00:00, 15.71it/s]


### 차로변경차량 정보추출

In [7]:
LCTG = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/LC_Seoul_TG.csv'

In [10]:
df2 = pd.DataFrame()
    
# 서울TG 부산방향에서 차로변경을 한 차량 리스트만 뽑기
LC_veh_list = df[df['Lane_change'] == 'Change']['Vehicle ID'].unique()
    
for veh in tqdm(LC_veh_list): # 차로변경을 한 각 차량별로
    df_veh = df[df['Vehicle ID'] == veh] ## 각 차량별 데이터프레임
    df_veh['Lane_record_split'] = df_veh['Lane_record_split'].backfill()
    df_veh['Lane_present'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[0]
    df_veh['Lane_target'] = df_veh['Lane_record_split'].str.split('_', n = 1, expand = True)[1]
        
    df_veh['Lane_change_intend'] = df_veh['Lane_change_direction'].copy()
    df_veh['Lane_change_intend'] = df_veh['Lane_change_intend'].backfill()
    
    df2 = pd.concat([df2, df_veh])
    
df2.dropna(subset = ['Lane_record_split'], inplace = True) # Lane_record_split이 nan인 행 드랍
        
df2.to_csv(LCTG)

100%|████████████████████████████████████████████████████████████████████████████████| 219/219 [00:02<00:00, 77.23it/s]
